In [ ]:
import numpy as np
import pandas as pd
from typing import List, Tuple, Optional
import os
import matplotlib.pyplot as plt
from scipy import signal
import cv2
from scipy.stats import entropy
from scipy.spatial import ConvexHull
import japanize_matplotlib
import seaborn as sns
from typing import List, Tuple, Dict
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pyVHR as vhr
from pyVHR.extraction.sig_processing import SignalProcessing
from pyVHR.plot.visualize import *
from pyVHR.BVP import *
vhr.plot.VisualizeParams.renderer = 'notebook'
from scipy.stats import skew, kurtosis, entropy
from skimage.metrics import structural_similarity as ssim
from scipy.fft import fft, fftfreq

# 修正: scikit-image 0.18.3用のインポート
from skimage.feature import local_binary_pattern
# graycomatrixとgraycopropsは古いバージョンでは別の場所にあります
try:
    from skimage.feature import greycomatrix as graycomatrix, greycoprops as graycoprops
except ImportError:
    # 別の方法を試す
    from skimage.feature import texture
    # または関数を使わない場合はコメントアウト
    graycomatrix = None
    graycoprops = None
    print("警告: graycomatrix/graycopropsをインポートできませんでした")

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from scipy.ndimage import laplace
import pywt
from sklearn.decomposition import PCA

In [ ]:
# 入力とする動画と動画のファイル名を取得
root_dir = "experimentData\\"
data_dirs = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
movie_paths = []
movie_names = []
true_value_csv_array = []
true_value_rri_csv_array = []
print("動画ディレクトリ:", data_dirs)

for i in range(len(data_dirs)):
    data_dir = data_dirs[i]

    # 動画ファイルのパスを取得
    movie_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.avi')]
    movie_paths.extend(movie_files)

    movie_name = os.path.basename(data_dir)
    movie_names.append(movie_name)

    # ppgファイルのパスを取得
    movie_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
    true_value_csv_array = [f.replace('.avi', '.csv') for f in movie_paths if f.endswith('.avi')]
    true_value_rri_csv_array.append(os.path.join(data_dir, 'RRI_Simple_' + movie_name + '.csv'))


f_1_ffi = 0.0399  # LFのはじめ
f_2 = 0.151  # LFの終わり、HFのはじめ
f_3 = 0.401  # HFの終わり

# start_index = 9
# end_index = len(movie_paths)

# data_dirs = data_dirs[start_index:end_index]
# movie_paths = movie_paths[start_index:end_index]
# movie_names = movie_names[start_index:end_index]
# true_value_csv_array = true_value_csv_array[start_index:end_index]
# true_value_rri_csv_array = true_value_rri_csv_array[start_index:end_index]

print(f"data_dirs: {data_dirs}")
print(f"movie_paths: {movie_paths}")
print(f"movie_names: {movie_names}")
print(f"true_value_csv_array: {true_value_csv_array}")
print(f"true_value_rri_csv_array: {true_value_rri_csv_array}")

In [ ]:
SAVE_DIR  = "LandmarksAccuracyEvalResults"
# rPPGに最適なランドマーク番号を定義

# 代表的な部分のみ
# FOREHEAD_LANDMARKS = [151]  # 額
UPPER_LEFT_CHEEK_LANDMARKS = [119]  # 左頬
LOWER_LEFT_CHEEK_LANDMARKS = [207]  # 左頬
UPPER_RIGHT_CHEEK_LANDMARKS = [347]  # 右頬
LOWER_RIGHT_CHEEK_LANDMARKS = [425]  # 右頬
NOSE_LANDMARKS = [4]
CHIN_LANDMARKS = [200]  # 顎

# ALL_RPPG_LANDMARKS = FOREHEAD_LANDMARKS + UPPER_LEFT_CHEEK_LANDMARKS + LOWER_LEFT_CHEEK_LANDMARKS + UPPER_RIGHT_CHEEK_LANDMARKS + LOWER_RIGHT_CHEEK_LANDMARKS + NOSE_LANDMARKS + CHIN_LANDMARKS
ALL_RPPG_LANDMARKS = UPPER_LEFT_CHEEK_LANDMARKS + LOWER_LEFT_CHEEK_LANDMARKS + UPPER_RIGHT_CHEEK_LANDMARKS + LOWER_RIGHT_CHEEK_LANDMARKS + NOSE_LANDMARKS + CHIN_LANDMARKS
# PATCH_SIZE = 10.0  # ランドマーク周辺のBoxelの一辺の長さ（ピクセル数）
PATCH_SIZE = 30.0  # ランドマーク周辺のBoxelの一辺の長さ（ピクセル数）
# PATCH_SIZE = 50.0  # ランドマーク周辺のBoxelの一辺の長さ（ピクセル数）

LANDMARK_ID_NAME_MAP = {
    151: "Forehead",
    119: "Upper Left Cheek",
    207: "Lower Left Cheek",
    347: "Upper Right Cheek",
    425: "Lower Right Cheek",
    4: "Nose",
    200: "Chin"
}

GLOBAL_SAVE_DIR = root_dir
print(f"GLOBAL_SAVE_DIR: {GLOBAL_SAVE_DIR}")

ランドマークの座標を記録
- {dataName}_landmarks_coordinates.csvを作成

In [ ]:
for movie_idx in range(len(movie_paths)):
    inputMoviePath = movie_paths[movie_idx]
    rootDir = data_dirs[movie_idx]
    dataName = movie_names[movie_idx]
    saveDir = os.path.join(rootDir, SAVE_DIR)
    print(f"ランドマーク抽出結果保存ディレクトリ: {saveDir}")
    os.makedirs(saveDir, exist_ok=True)
    print(f'Processing movie: {inputMoviePath}')
    
    # 動画のfpsを取得
    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    samplingRate = fps
    
    # 1フレーム目を読み込む
    ret, first_frame = cap.read()
    cap.release()
    
    if not ret:
        print(f"Error: 最初のフレームを読み込めませんでした: {inputMoviePath}")
        continue
    
    # === Step 1: ランドマーク抽出用のSignalProcessing初期化 ===
    print("=== ランドマーク抽出開始 ===")
    sig_processing = SignalProcessing()
    sig_processing.set_landmarks(ALL_RPPG_LANDMARKS)
    sig_processing.set_square_patches_side(PATCH_SIZE)
    
    # 可視化を有効化
    sig_processing.set_visualize_skin_and_landmarks(
        visualize_skin=True,
        visualize_landmarks=True,
        visualize_landmarks_number=True,
        visualize_patch=True
    )
    
    # 肌抽出器を設定(ConvexHullを使用)
    sig_processing.set_skin_extractor(vhr.extraction.SkinExtractionConvexHull('CPU'))
    
    # 1フレームのみ処理
    sig_processing.set_total_frames(1)
    
    # ダミーでextract_patchesを実行してランドマーク検出を行う
    try:
        _ = sig_processing.extract_patches(inputMoviePath, "squares", "mean")
        
        # 可視化画像を取得
        visualize_patches = sig_processing.get_visualize_patches()
        
        if len(visualize_patches) > 0:
            # 1フレーム目の可視化画像を保存
            output_path = os.path.join(saveDir, f"{dataName}_landmarks_frame1.png")
            cv2.imwrite(output_path, visualize_patches[0])
            print(f"ランドマーク画像を保存しました: {output_path}")
            
            # ランドマークの座標を取得して保存
            # MediaPipeを使用してランドマークを検出
            import mediapipe as mp
            mp_face_mesh = mp.solutions.face_mesh
            
            with mp_face_mesh.FaceMesh(
                static_image_mode=True,
                max_num_faces=1,
                refine_landmarks=True,
                min_detection_confidence=0.5) as face_mesh:
                
                # RGB変換
                frame_rgb = cv2.cvtColor(first_frame, cv2.COLOR_BGR2RGB)
                results = face_mesh.process(frame_rgb)
                
                if results.multi_face_landmarks:
                    landmarks_data = []
                    face_landmarks = results.multi_face_landmarks[0]
                    h, w = first_frame.shape[:2]
                    
                    # 使用したランドマークの座標を取得
                    for idx in ALL_RPPG_LANDMARKS:
                        landmark = face_landmarks.landmark[idx]
                        x = int(landmark.x * w)
                        y = int(landmark.y * h)
                        landmarks_data.append({
                            'landmark_id': idx,
                            'x': x,
                            'y': y
                        })
                    
                    # CSVファイルに保存
                    import pandas as pd
                    df = pd.DataFrame(landmarks_data)
                    csv_path = os.path.join(saveDir, f"{dataName}_landmarks_coordinates.csv")
                    df.to_csv(csv_path, index=False)
                    print(f"ランドマーク座標を保存しました: {csv_path}")
                else:
                    print(f"Warning: 顔のランドマークが検出されませんでした: {inputMoviePath}")
        else:
            print(f"Warning: ランドマークが検出されませんでした: {inputMoviePath}")
            
    except Exception as e:
        print(f"Error: ランドマーク検出中にエラーが発生しました: {e}")
        continue
    
    print(f"=== {dataName} の処理完了 ===\n")

    # ここにランドマークのidと座標を保存するコードを追加

記録された座標を読み込み、各々のRGBを記録

In [ ]:
def make_analysis_df(video_path, roi_coordinates, fps):
    """
    RGB + HSV + L*a*b*特徴量を含む画像品質解析を行う関数
    
    Args:
        video_path: 動画ファイルのパス
        roi_coordinates: ROI座標 (left, top, right, bottom)
        fps: フレームレート
    
    Returns:
        DataFrame: 解析結果
    """
    cap = cv2.VideoCapture(video_path)
    
    # 結果を格納するリスト
    results = {
        'frame_number': [],
        'timestamp': [],
        'contrast': [],
        'r_mean': [],
        'g_mean': [],
        'b_mean': [],
        'r_std': [],
        'g_std': [],
        'b_std': [],
        's_mean': [],
        's_std': [],
        'l_mean': [],
        'l_std': [],
    }
    
    frame_count = 0
    roi_left, roi_top, roi_right, roi_bottom = roi_coordinates
    
    print(f"動画解析開始: {video_path}")
    print(f"ROI座標: ({roi_left}, {roi_top}, {roi_right}, {roi_bottom})")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # ROI領域を切り出し
        roi_frame = frame[roi_top:roi_bottom, roi_left:roi_right]
        
        if roi_frame.size == 0:
            continue
            
        # タイムスタンプ計算
        timestamp = frame_count / fps
        
        # RGB解析
        b_channel, g_channel, r_channel = cv2.split(roi_frame)
        
        r_mean = np.mean(r_channel)
        g_mean = np.mean(g_channel)
        b_mean = np.mean(b_channel)
        
        r_std = np.std(r_channel)
        g_std = np.std(g_channel)
        b_std = np.std(b_channel)
        
        # コントラスト計算（グレースケール）
        gray_roi = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)
        contrast = np.std(gray_roi)
        
        # HSV解析
        hsv_roi = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2HSV)
        h_channel, s_channel, v_channel = cv2.split(hsv_roi)
        
        s_mean = np.mean(s_channel)
        s_std = np.std(s_channel)
        
        # L*a*b*解析
        lab_roi = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2LAB)
        l_channel, a_channel, b_channel_lab = cv2.split(lab_roi)
        
        l_mean = np.mean(l_channel)
        l_std = np.std(l_channel)
        
        # 結果を保存
        results['frame_number'].append(frame_count)
        results['timestamp'].append(timestamp)
        results['contrast'].append(contrast)
        results['r_mean'].append(r_mean)
        results['g_mean'].append(g_mean)
        results['b_mean'].append(b_mean)
        results['r_std'].append(r_std)
        results['g_std'].append(g_std)
        results['b_std'].append(b_std)
        results['s_mean'].append(s_mean)
        results['s_std'].append(s_std)
        results['l_mean'].append(l_mean)
        results['l_std'].append(l_std)
        
        frame_count += 1
        
        # 進捗表示
        if frame_count % 100 == 0:
            print(f"処理済みフレーム: {frame_count}")
    
    cap.release()
    
    # DataFrameに変換
    df = pd.DataFrame(results)
    print(f"解析完了: {len(df)} フレーム処理")
    
    return df

In [ ]:
for movie_idx in range(len(movie_paths)):  # ★ 変数名を変更
    inputMoviePath = movie_paths[movie_idx]
    rootDir = data_dirs[movie_idx]
    dataName = movie_names[movie_idx]
    
    print(f'Processing movie: {dataName}')  # ★ デバッグ用
    
    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    savedir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(savedir, exist_ok=True)
    
    # ROIの座標を取得
    roi_coordinates_path = os.path.join(savedir, f"{dataName}_landmarks_coordinates.csv")
    roi_df = pd.read_csv(roi_coordinates_path)
    
    # ★★★ 動画ごとにリストとDataFrameを初期化 ★★★
    all_segments_data = []
    all_segments_df = pd.DataFrame()
    
    # ROIごとにRGB、HSVを記録
    for roi_idx, row in roi_df.iterrows():
        landmark_id = int(row['landmark_id'])  # 整数に変換
        center_x = int(row['x'])  # 整数に変換
        center_y = int(row['y'])  # 整数に変換
        
        # パッチの座標を計算（中心座標から±PATCH_SIZE/2）
        half_patch = PATCH_SIZE // 2
        landmark_left = int(max(0, center_x - half_patch))
        landmark_top = int(max(0, center_y - half_patch))
        landmark_right = int(center_x + half_patch)
        landmark_bottom = int(center_y + half_patch)
        
        roi = (landmark_left, landmark_top, landmark_right, landmark_bottom)
        
        print(f'  Processing landmark_id: {landmark_id} at ({center_x}, {center_y})')
        
        landmark_df = make_analysis_df(
            video_path=inputMoviePath,
            roi_coordinates=roi,
            fps=fps
        )
        landmark_df["landmark_id"] = landmark_id
        print(f'  landmark {landmark_id} data shape: {landmark_df.shape}')
        
        # 動画のデータをリストに追加
        all_segments_data.append(landmark_df)
    
    # 全てのランドマークのデータを統合
    all_landmarks_df = pd.concat(all_segments_data, ignore_index=True)
    print(f'Total data shape for {dataName}: {all_landmarks_df.shape}')
    
    # CSVとして保存
    output_csv_path = os.path.join(rootDir, SAVE_DIR, f"{dataName}_{PATCH_SIZE}x{PATCH_SIZE}_landmarks_data.csv")
    all_landmarks_df.to_csv(output_csv_path, index=False)
    print(f'Saved: {output_csv_path}')
    
    # === ROI可視化 ===
    # 1フレーム目を読み込み
    cap = cv2.VideoCapture(inputMoviePath)
    ret, first_frame = cap.read()
    cap.release()
    
    if ret:
        vis_frame = first_frame.copy()
        
        # CSVから各ランドマークのROIを描画
        for roi_idx, row in roi_df.iterrows():
            landmark_id = int(row['landmark_id'])
            center_x = int(row['x'])
            center_y = int(row['y'])
            
            # パッチの座標を計算
            half_patch = PATCH_SIZE // 2
            left = int(max(0, center_x - half_patch))
            top = int(max(0, center_y - half_patch))
            right = int(center_x + half_patch)
            bottom = int(center_y + half_patch)
            
            # ROIの矩形を描画（緑色）
            cv2.rectangle(vis_frame, (left, top), (right, bottom), (0, 255, 0), 2)
            
            # ランドマークIDを表示
            cv2.putText(vis_frame, str(landmark_id), (center_x - 10, center_y - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
            # 中心点を描画（赤色）
            cv2.circle(vis_frame, (center_x, center_y), 3, (0, 0, 255), -1)
        
        # 可視化画像を保存
        vis_output_path = os.path.join(savedir, f"{dataName}_{PATCH_SIZE}x{PATCH_SIZE}_ROI_visualization.png")
        cv2.imwrite(vis_output_path, vis_frame)
        print(f'ROI可視化画像を保存: {vis_output_path}\n')
    else:
        print(f'Error: フレームの読み込みに失敗しました\n')

各窓にRGB信号を格納

In [ ]:
WINDOW_SIZES = [2, 4, 6, 8, 10]
STRIDES = [2, 4]

In [ ]:
def array_to_full_string(arr):
    """NumPy配列を省略なしの文字列に変換"""
    if isinstance(arr, str):
        return arr
    elif isinstance(arr, (np.ndarray, list)):
        # NumPy配列またはリストを完全な文字列に変換
        arr_np = np.array(arr)
        return np.array2string(arr_np, threshold=np.inf, max_line_width=np.inf, separator=' ')
    else:
        return str(arr)

class StrideSegmentCalculator:
    def __init__(
        self,
        window_sizes: List[float] = [2, 3, 4, 5],
        strides: List[float] = [0.1, 0.5, 1, 1.5, 2],
    ):
        """
        Parameters:
        -----------
        window_sizes : List[float]
            窓幅（秒）のリスト
        strides : List[float]
            移動秒数のリスト
        """
        self.window_sizes = window_sizes
        self.strides = strides

    def calculate_overlap(self, window_size: float, stride: float) -> float:
        """
        窓幅と移動秒数からオーバーラップ率を計算

        Parameters:
        -----------
        window_size : float
            窓幅（秒）
        stride : float
            移動秒数（秒）

        Returns:
        --------
        float
            オーバーラップ率（%）
        """
        if stride >= window_size:
            return 0
        overlap = (window_size - stride) / window_size * 100
        return round(overlap, 2)

    def calculate_segments(
        self, window_size: float, stride: float, total_frames: int, fps: int
    ) -> List[Tuple[int, int]]:
        """
        フレーム数から解析区間を計算

        Parameters:
        -----------
        window_size : float
            窓幅（秒）
        stride : float
            移動秒数（秒）
        total_frames : int
            総フレーム数
        fps : int
            フレームレート

        Returns:
        --------
        List[Tuple[int, int]]
            各区間の(開始フレーム, 終了フレーム)のリスト
        """
        frames_per_window = round(window_size * fps)
        frames_per_stride = round(stride * fps)

        segments = []
        start_frame = 0

        while start_frame + frames_per_window <= total_frames:
            segments.append((start_frame, start_frame + frames_per_window))
            start_frame += frames_per_stride

        return segments

    def create_analysis_dataframe(self, total_frames: int, fps: int) -> pd.DataFrame:
        """
        全ての窓幅と移動秒数の組み合わせに対してDataFrameを生成

        Parameters:
        -----------
        total_frames : int
            総フレーム数
        fps : int
            フレームレート

        Returns:
        --------
        pd.DataFrame
            各条件でのセグメント情報を含むDataFrame
            columns: window_size, stride, overlap, segment_number, frame_start, frame_end
        """
        data_dict = {
            "window_size": [],
            "stride": [],
            "overlap": [],
            "segment_number": [],
            "frame_start": [],
            "frame_end": [],
        }

        for window_size in self.window_sizes:
            for stride in self.strides:
                overlap = self.calculate_overlap(window_size, stride)
                segments = self.calculate_segments(
                    window_size, stride, total_frames, fps
                )

                for i, (start_frame, end_frame) in enumerate(segments):
                    data_dict["window_size"].append(window_size)
                    data_dict["stride"].append(stride)
                    data_dict["overlap"].append(overlap)
                    data_dict["segment_number"].append(i)
                    data_dict["frame_start"].append(start_frame)
                    data_dict["frame_end"].append(end_frame)

        return pd.DataFrame(data_dict)


class PulseAnalysisDataStrides:
    def __init__(self, window_sizes, strides):
        # 窓枠とストライドの値を定義
        self.window_sizes = window_sizes
        self.strides = strides

        # accuracyのみを格納するDataFrameを初期化
        self.results = pd.DataFrame(
            index=pd.Index(self.window_sizes, name="window_size"),
            columns=pd.Index(self.strides, name="strides"),
        )

    def add_accuracy(self, window_size: float, strides: int, accuracy: float):
        """
        精度データを追加する

        Parameters:
        -----------
        window_size : float
            窓幅（秒）
        strides : int
            ストライド（s）
        accuracy : float
            精度値
        """
        self.results.loc[window_size, strides] = accuracy

    def _create_heatmap_dataframe(self):
        """
        ヒートマップ用のDataFrameを作成する内部メソッド

        Returns:
        --------
        pd.DataFrame
            ヒートマップ用に整形されたDataFrame
        """
        data = {"stride": self.strides}
        for window_size in self.window_sizes:
            data[window_size] = [
                self.results.loc[window_size, stride] for stride in self.strides
            ]
        df = pd.DataFrame(data).set_index("stride").T
        return df

    def save_heatmap(
        self,
        title: str,
        save_path: str,
        figsize: tuple = (10, 8),
        cmap: str = "YlGnBu",
        colorbar_label: str = "MAE",
    ):
        """
        ヒートマップを作成して保存する

        Parameters:
        -----------
        title : str
            プロットのタイトル
        save_path : str
            保存先のパス
        figsize : tuple, optional
            図のサイズ (default: (10, 8))
        cmap : str, optional
            カラーマップ (default: 'YlGnBu')
        colorbar_label : str, optional
            カラーバーのラベル (default: 'MAE')
        """
        df = self._create_heatmap_dataframe()

        # ヒートマップを作成
        plt.figure(figsize=figsize)
        sns.heatmap(
            df, annot=True, fmt=".4f", cmap=cmap, cbar_kws={"label": colorbar_label}
        )
        plt.title(f"{title}")
        plt.xlabel("Stride [s]")
        plt.ylabel("Window Size [s]")
        plt.tight_layout()
        plt.savefig(save_path, dpi=300)
        plt.close()

    def save_heatmap_std(self, title: str, save_path: str, figsize: tuple = (10, 8)):
        """
        標準偏差のヒートマップを作成して保存する

        Parameters:
        -----------
        title : str
            プロットのタイトル
        save_path : str
            保存先のパス
        figsize : tuple, optional
            図のサイズ (default: (10, 8))
        """
        self.save_heatmap(
            title,
            save_path,
            figsize,
            cmap="Reds",
            colorbar_label="Standard Deviation",
        )

In [ ]:
for i in range(len(movie_paths)):
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    print(f'Processing movie: {inputMoviePath}')

    # 動画のfpsを取得
    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    samplingRate = fps
    cap.release()

    save_dir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(save_dir, exist_ok=True)        
    

    # CSVファイルの読み込み
    ecg_csv_path = os.path.join(rootDir, dataName + '.csv')
    ecg_df = pd.read_csv(ecg_csv_path)

    ecg_RRI_csv_path = os.path.join(rootDir, f'RRI_Simple_{movie_names[i]}.csv')
    ecg_RRI_df = pd.read_csv(ecg_RRI_csv_path)

    # landmarkで共通のecg_bpm_in_window_meanを取得
    ecg_bpm_in_window = ecg_RRI_df['BPM']
    ecg_bpm_in_window_mean = ecg_RRI_df['BPM'].values.mean()

    # ROIデータの読み込み
    os.makedirs(save_dir, exist_ok=True)
    landmark_data_path = os.path.join(rootDir, SAVE_DIR, f"{dataName}_{PATCH_SIZE}x{PATCH_SIZE}_landmarks_data.csv")
    landmark_data_df = pd.read_csv(landmark_data_path)

    stride_segment_calculator = StrideSegmentCalculator(window_sizes=WINDOW_SIZES, strides=STRIDES)
    analysis_df = stride_segment_calculator.create_analysis_dataframe(total_frames, fps)

    # ランドマークごとに実行
    # 結果を格納するリスト
    all_window_results = []
    for idx, row in analysis_df.iterrows():
        window_size = row['window_size']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        stride = row['stride']

        # 窓の時間範囲を計算
        window_start_time = frame_start / fps
        window_end_time = frame_end / fps

        if(window_end_time > total_frames / fps):
            continue

        print(f"\nウィンドウ {idx}: 窓サイズ {window_size}s, ストライド {stride}s, フレーム {frame_start}-{frame_end}, 時間 {window_start_time:.2f}-{window_end_time:.2f}s")

        # 該当する窓の時間範囲内の真値RRIデータを抽出
        ecg_RRI_mask = (ecg_RRI_df['time'] >= window_start_time) & \
                (ecg_RRI_df['time'] < window_end_time)
        ecg_bpm_in_window = ecg_RRI_df[ecg_RRI_mask]['BPM'].values
        ecg_bpm_in_window_mean = np.mean(ecg_bpm_in_window) if len(ecg_bpm_in_window) > 0 else np.nan
        print(f'    True Value RRI count in window: {len(ecg_bpm_in_window)}, Mean BPM: {ecg_bpm_in_window_mean:.2f}')
        
        for landmark_id in landmark_data_df['landmark_id'].unique():
            print(f'  Analyzing landmark_id: {landmark_id}')
            landmark_df = landmark_data_df[landmark_data_df['landmark_id'] == landmark_id]
            print(f'    landmark data shape: {landmark_df}')
            

            # 該当する窓の時間範囲内のRGB信号を抽出し、窓内でBVP算出
            bvp_mask = (landmark_df['timestamp'] >= window_start_time) & (landmark_df['timestamp'] < window_end_time)

            r_signal_in_window = landmark_df[bvp_mask]['r_mean'].values
            g_signal_in_window = landmark_df[bvp_mask]['g_mean'].values
            b_signal_in_window = landmark_df[bvp_mask]['b_mean'].values

            saturation_signal_in_window = landmark_df[bvp_mask]['s_mean'].values
            lightness_signal_in_window = landmark_df[bvp_mask]['l_mean'].values

            # 窓情報を保存
            window_info = {
                'landmark_id': landmark_id,
                'window_index': idx,
                'window_size': window_size,
                'stride': stride,
                'frame_start': frame_start,
                'frame_end': frame_end,
                'window_start_time': window_start_time,
                'window_end_time': window_end_time,
                'r_signal_in_window': array_to_full_string(r_signal_in_window),
                'g_signal_in_window': array_to_full_string(g_signal_in_window),
                'b_signal_in_window': array_to_full_string(b_signal_in_window),
                'saturation_signal_in_window': array_to_full_string(saturation_signal_in_window),
                'lightness_signal_in_window': array_to_full_string(lightness_signal_in_window)
            }
            all_window_results.append(window_info)
    # 全結果をDataFrameに変換して保存
    results_df = pd.DataFrame(all_window_results)
    results_save_dir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(results_save_dir, exist_ok=True)
    results_csv_path = os.path.join(results_save_dir, f'{dataName}_{PATCH_SIZE}x{PATCH_SIZE}_ROI_window_signals.csv')
    results_df.to_csv(results_csv_path, index=False, encoding='utf-8-sig')
    print(f"\nFFT結果をCSVに保存: {results_csv_path}")

### 窓ごとにBVPとMAEを算出し、保存

In [ ]:
def extract_BVPsignal(r_signals, g_signals, b_signals,  fps, deviceType, bvpMethod, bvpMethodName, method_params=None):
    """
    RGB信号からBVP信号を抽出する関数
    """
    rgb_signal = np.array([[r_signals, g_signals, b_signals]], dtype=np.float32)
    print(f"\nRGB信号の形状: {rgb_signal.shape}")
    
    signal_length = rgb_signal.shape[2]
    min_required_length = 50
    
    if signal_length < min_required_length:
        print(f"警告: 信号長が短すぎます ({signal_length} < {min_required_length})。処理をスキップします。")
        return None, None
    
    filtered_signal = [rgb_signal]
    
    # デフォルトのメソッドパラメータ
    if method_params is None:
        method_params = {}
    
    # メソッド別パラメータ設定
    if bvpMethodName in ["cupy_POS", "cpu_POS"]:
        method_params['fps'] = fps
    elif bvpMethodName in ["cpu_ICA", "cpu_PCA"]:
        method_params['component'] = 'all_comp'
    
    print(f"\nBVP抽出開始 (メソッド: {bvpMethodName})")
    print(f"パラメータ: {method_params}")
    
    # BVP信号抽出
    if method_params:
        bvp_signal = vhr.BVP.RGB_sig_to_BVP(
            filtered_signal,
            fps,
            device_type=deviceType,
            method=bvpMethod,
            params=method_params
        )
    else:
        bvp_signal = vhr.BVP.RGB_sig_to_BVP(
            filtered_signal,
            fps,
            device_type=deviceType,
            method=bvpMethod
        )
    
    # 生のBVP信号を保存
    raw_bvp_signal = bvp_signal[0].copy() if len(bvp_signal) > 0 else None
    
    # 後処理フィルタリング
    bvp_signal = vhr.BVP.apply_filter(
        bvp_signal,
        vhr.BVP.BPfilter,
        params={'order': 6, 'minHz': 0.5, 'maxHz': 2.0, 'fps': fps}
    )
    
    bvp_signal = vhr.BVP.apply_filter(bvp_signal, vhr.BVP.zeromean)
    
    filtered_bvp_signal = bvp_signal[0] if len(bvp_signal) > 0 else None
    
    print(f"\nBVP信号抽出完了")
    return raw_bvp_signal, filtered_bvp_signal

In [ ]:
def analyze_window_fft(values, fps):
    """
    時系列データの最大周波数とスペクトル情報を計算する関数

    Parameters:
    -----------
    values : array-like
        分析対象の時系列データ
    fps : int
        サンプリング周波数（1秒あたりのフレーム数）

    Returns:
    --------
    dict
        以下のキーを含む辞書：
        - 'max_freq': 検出された最大周波数
        - 'max_amplitude': 最大周波数のときの振幅
        - 'frequencies': 周波数配列（正の周波数のみ）
        - 'amplitudes': 振幅配列（正の周波数のみ）
        - 'power_spectrum': パワースペクトル
        - 'dominant_freqs': 上位5つの卓越周波数とその振幅
        - 'spectral_centroid': スペクトル重心
        - 'spectral_bandwidth': スペクトル帯域幅
        - 'total_power': 全体のパワー
    """
    # データをnumpy配列に変換
    values = np.array(values)

    # ゼロパディングで分解能を向上（窓長の8倍）
    n_pad = len(values) * 8

    # ハミング窓を適用（オプション：コメントアウトされている）
    # window = np.hamming(len(values))
    # windowed_data = values * window

    # FFTを実行（ゼロパディング適用）
    fft_result = np.fft.fft(values, n=n_pad)
    fft_freq = np.fft.fftfreq(n_pad, 1 / fps)

    # 正の周波数のみを取得
    positive_freq_idx = fft_freq > 0
    positive_fft = np.abs(fft_result[positive_freq_idx])
    positive_freq = fft_freq[positive_freq_idx]
    
    # パワースペクトルを計算
    power_spectrum = positive_fft ** 2

    # 最大周波数の検出と補間
    max_idx = np.argmax(positive_fft)
    max_amplitude = positive_fft[max_idx]
    
    if 0 < max_idx < len(positive_fft) - 1:
        # 3点を使用した放物線補間
        alpha = positive_fft[max_idx - 1]
        beta = positive_fft[max_idx]
        gamma = positive_fft[max_idx + 1]
        peak_pos = 0.5 * (alpha - gamma) / (alpha - 2 * beta + gamma)

        # 補間された周波数と振幅
        freq_resolution = fps / n_pad
        max_freq = positive_freq[max_idx] + peak_pos * freq_resolution
        
        # 補間された振幅（放物線の頂点）
        max_amplitude = beta - 0.25 * (alpha - gamma) * peak_pos
    else:
        max_freq = positive_freq[max_idx]
    
    # 上位5つの卓越周波数を検出
    top_indices = np.argsort(positive_fft)[-5:][::-1]
    dominant_freqs = [(positive_freq[idx], positive_fft[idx]) for idx in top_indices]
    
    # スペクトル特徴量の計算
    # スペクトル重心（周波数の重み付き平均）
    spectral_centroid = np.sum(positive_freq * positive_fft) / np.sum(positive_fft)
    
    # スペクトル帯域幅（重心からの重み付き分散）
    spectral_bandwidth = np.sqrt(
        np.sum(((positive_freq - spectral_centroid) ** 2) * positive_fft) / np.sum(positive_fft)
    )
    
    # 全体のパワー
    total_power = np.sum(power_spectrum)
    
    # 結果を辞書にまとめる
    result = {
        'max_freq': max_freq,
        'max_amplitude': max_amplitude,
        'frequencies': positive_freq,
        'amplitudes': positive_fft,
        'power_spectrum': power_spectrum,
        'dominant_freqs': dominant_freqs,
        'spectral_centroid': spectral_centroid,
        'spectral_bandwidth': spectral_bandwidth,
        'total_power': total_power,
        'freq_resolution': fps / n_pad,  # 周波数分解能
        'nyquist_freq': fps / 2  # ナイキスト周波数
    }
    
    return result

In [ ]:
for i in range(len(movie_paths)):
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    print(f'Processing movie: {inputMoviePath}')

    # 動画のfpsを取得
    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    samplingRate = fps
    cap.release()

    save_dir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(save_dir, exist_ok=True)        

    # CSVファイルの読み込み
    ecg_csv_path = os.path.join(rootDir, dataName + '.csv')
    ecg_df = pd.read_csv(ecg_csv_path)

    ecg_RRI_csv_path = os.path.join(rootDir, f'RRI_Simple_{movie_names[i]}.csv')
    ecg_RRI_df = pd.read_csv(ecg_RRI_csv_path)

    # landmarkで共通のecg_bpm_in_window_meanを取得
    ecg_bpm_in_window = ecg_RRI_df['BPM']
    ecg_bpm_in_window_mean = ecg_RRI_df['BPM'].values.mean()

    # ROIデータの読み込み
    os.makedirs(save_dir, exist_ok=True)
    window_signals_data_path = os.path.join(rootDir, SAVE_DIR, f'{dataName}_{PATCH_SIZE}x{PATCH_SIZE}_ROI_window_signals.csv')
    window_signals_data_df = pd.read_csv(window_signals_data_path)

    stride_segment_calculator = StrideSegmentCalculator(window_sizes=WINDOW_SIZES, strides=STRIDES)
    analysis_df = stride_segment_calculator.create_analysis_dataframe(total_frames, fps)

    # ランドマークごとに実行
    # 結果を格納するリスト
    all_window_results = []
    for idx, row in analysis_df.iterrows():
        window_size = row['window_size']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        stride = row['stride']

        # 窓の時間範囲を計算
        window_start_time = frame_start / fps
        window_end_time = frame_end / fps

        if(window_end_time > total_frames / fps):
            continue

        print(f"\nウィンドウ {idx}: 窓サイズ {window_size}s, ストライド {stride}s, フレーム {frame_start}-{frame_end}, 時間 {window_start_time:.2f}-{window_end_time:.2f}s")

        # 該当する窓の時間範囲内の真値RRIデータを抽出
        ecg_RRI_mask = (ecg_RRI_df['time'] >= window_start_time) & \
                (ecg_RRI_df['time'] < window_end_time)
        ecg_bpm_in_window = ecg_RRI_df[ecg_RRI_mask]['BPM'].values
        ecg_bpm_in_window_mean = np.mean(ecg_bpm_in_window) if len(ecg_bpm_in_window) > 0 else np.nan
        print(f'    True Value RRI count in window: {len(ecg_bpm_in_window)}, Mean BPM: {ecg_bpm_in_window_mean:.2f}')
        
        for landmark_id in window_signals_data_df['landmark_id'].unique():
            print(f'  Analyzing landmark_id: {landmark_id}')
            landmark_df = window_signals_data_df[window_signals_data_df['landmark_id'] == landmark_id]

            # 該当する窓の時間範囲内のRGB信号を抽出し、窓内でBVP算出
            bvp_mask = landmark_df['window_index'] == idx

            r_signal_in_window = landmark_df[bvp_mask]['r_signal_in_window'].values
            g_signal_in_window = landmark_df[bvp_mask]['g_signal_in_window'].values
            b_signal_in_window = landmark_df[bvp_mask]['b_signal_in_window'].values
            saturation_signal_in_window = landmark_df[bvp_mask]['saturation_signal_in_window'].values
            lightness_signal_in_window = landmark_df[bvp_mask]['lightness_signal_in_window'].values

            # 文字列をNumPy配列に変換
            r_signal_in_window = np.fromstring(r_signal_in_window[0].strip('[]'), sep=' ') if len(r_signal_in_window) > 0 else np.array([])
            g_signal_in_window = np.fromstring(g_signal_in_window[0].strip('[]'), sep=' ') if len(g_signal_in_window) > 0 else np.array([])
            b_signal_in_window = np.fromstring(b_signal_in_window[0].strip('[]'), sep=' ') if len(b_signal_in_window) > 0 else np.array([])
            saturation_signal_in_window = np.fromstring(saturation_signal_in_window[0].strip('[]'), sep=' ') if len(saturation_signal_in_window) > 0 else np.array([])
            lightness_signal_in_window = np.fromstring(lightness_signal_in_window[0].strip('[]'), sep=' ') if len(lightness_signal_in_window) > 0 else np.array([])

            print(f'    RGB信号の長さ: R={len(r_signal_in_window)}, G={len(g_signal_in_window)}, B={len(b_signal_in_window)}')

            # BVPメソッドの設定
            methodCombinations =  ['cuda', cupy_POS, "cupy_POS"]
            deviceType = methodCombinations[0]  # 'cuda'
            bvpMethod = methodCombinations[1]   # cupy_POS
            bvpMethodName = methodCombinations[2]  # "cupy_POS"

            # rgbからBVPを計算
            raw_bvp_signal_in_window, filtered_bvp_signal_in_window = extract_BVPsignal(
                r_signal_in_window,
                g_signal_in_window,
                b_signal_in_window,
                fps,
                deviceType,
                bvpMethod,
                bvpMethodName
            )
            
            # BVP信号の抽出に失敗した場合はスキップ
            if filtered_bvp_signal_in_window is None:
                print(f"ウィンドウ {idx} をスキップ: BVP信号の抽出に失敗")
                continue
            
            # FFT解析
            raw_bvp_signal_in_window = raw_bvp_signal_in_window.flatten() if raw_bvp_signal_in_window is not None else None
            filtered_bvp_signal_in_window = filtered_bvp_signal_in_window.flatten()
            fft_result_dic = analyze_window_fft(filtered_bvp_signal_in_window, fps)

            # MAEの計算
            rppg_bpm = fft_result_dic['max_freq'] * 60
            rppg_freq = fft_result_dic['frequencies']
            rppg_amplitude = fft_result_dic['amplitudes']
            rppg_pwd = fft_result_dic['power_spectrum']

            bpm_MAE = np.abs(ecg_bpm_in_window_mean - rppg_bpm) if not np.isnan(ecg_bpm_in_window_mean) else np.nan

            # 窓情報を保存
            window_info = {
                'landmark_id': landmark_id,
                'window_index': idx,
                'window_size': window_size,
                'stride': stride,
                'frame_start': frame_start,
                'frame_end': frame_end,
                'window_start_time': window_start_time,
                'window_end_time': window_end_time,
                'r_signal_in_window': array_to_full_string(r_signal_in_window),
                'g_signal_in_window': array_to_full_string(g_signal_in_window),
                'b_signal_in_window': array_to_full_string(b_signal_in_window),
                'saturation_signal_in_window': array_to_full_string(saturation_signal_in_window),
                'lightness_signal_in_window': array_to_full_string(lightness_signal_in_window),
                'raw_bvp_in_window': raw_bvp_signal_in_window,
                'filtered_bvp_in_window': filtered_bvp_signal_in_window,
                'ecg_bpm_in_window': ecg_bpm_in_window,
                'ecg_bpm_mean': ecg_bpm_in_window_mean,
                'rppg_bpm': rppg_bpm,
                'bpm_MAE': bpm_MAE,
                'max_freq': fft_result_dic['max_freq'],
                'max_amplitude': fft_result_dic['max_amplitude'],
                'spectral_centroid': fft_result_dic['spectral_centroid'],
                'spectral_bandwidth': fft_result_dic['spectral_bandwidth'],
                'total_power': fft_result_dic['total_power']
            }
            all_window_results.append(window_info)
    # 全結果をDataFrameに変換して保存
    results_df = pd.DataFrame(all_window_results)
    results_save_dir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(results_save_dir, exist_ok=True)
    results_csv_path = os.path.join(results_save_dir, f'{dataName}_window_analysis_{PATCH_SIZE}x{PATCH_SIZE}.csv')
    results_df.to_csv(results_csv_path, index=False, encoding='utf-8-sig')
    print(f"\nFFT結果をCSVに保存: {results_csv_path}")

### ランドマークごとに輝度信号を算出

In [ ]:
import cupy as cp
for i in range(len(movie_paths)):
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    print(f'Processing movie: {inputMoviePath}')

    # 動画のfpsを取得
    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    samplingRate = fps
    cap.release()

    save_dir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(save_dir, exist_ok=True)        

    # CSVファイルの読み込み
    ecg_csv_path = os.path.join(rootDir, dataName + '.csv')
    ecg_df = pd.read_csv(ecg_csv_path)

    ecg_RRI_csv_path = os.path.join(rootDir, f'RRI_Simple_{movie_names[i]}.csv')
    ecg_RRI_df = pd.read_csv(ecg_RRI_csv_path)

    # landmarkで共通のecg_bpm_in_window_meanを取得
    ecg_bpm_in_window = ecg_RRI_df['BPM']
    ecg_bpm_in_window_mean = ecg_RRI_df['BPM'].values.mean()

    # ROIデータの読み込み
    os.makedirs(save_dir, exist_ok=True)
    window_signals_data_path = os.path.join(rootDir, SAVE_DIR, f'{dataName}_{PATCH_SIZE}x{PATCH_SIZE}_ROI_window_signals.csv')
    window_signals_data_df = pd.read_csv(window_signals_data_path)

    stride_segment_calculator = StrideSegmentCalculator(window_sizes=WINDOW_SIZES, strides=STRIDES)
    analysis_df = stride_segment_calculator.create_analysis_dataframe(total_frames, fps)

    # ランドマークごとに実行
    # 結果を格納するリスト
    all_window_results = []
    
    for idx, row in analysis_df.iterrows():
        window_size = row['window_size']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        stride = row['stride']

        # 窓の時間範囲を計算
        window_start_time = frame_start / fps
        window_end_time = frame_end / fps

        if(window_end_time > total_frames / fps):
            continue

        print(f"\nウィンドウ {idx}: 窓サイズ {window_size}s, ストライド {stride}s, フレーム {frame_start}-{frame_end}, 時間 {window_start_time:.2f}-{window_end_time:.2f}s")

        # 該当する窓の時間範囲内の真値RRIデータを抽出
        ecg_RRI_mask = (ecg_RRI_df['time'] >= window_start_time) & \
                (ecg_RRI_df['time'] < window_end_time)
        ecg_bpm_in_window = ecg_RRI_df[ecg_RRI_mask]['BPM'].values
        ecg_bpm_in_window_mean = np.mean(ecg_bpm_in_window) if len(ecg_bpm_in_window) > 0 else np.nan
        print(f'    True Value RRI count in window: {len(ecg_bpm_in_window)}, Mean BPM: {ecg_bpm_in_window_mean:.2f}')
        
        for landmark_id in window_signals_data_df['landmark_id'].unique():
            print(f'  Analyzing landmark_id: {landmark_id}')
            landmark_df = window_signals_data_df[window_signals_data_df['landmark_id'] == landmark_id]

            # 該当する窓の時間範囲内のRGB信号を抽出し、窓内でBVP算出
            bvp_mask = landmark_df['window_index'] == idx

            r_signal_in_window = landmark_df[bvp_mask]['r_signal_in_window'].values
            g_signal_in_window = landmark_df[bvp_mask]['g_signal_in_window'].values
            b_signal_in_window = landmark_df[bvp_mask]['b_signal_in_window'].values
            saturation_signal_in_window = landmark_df[bvp_mask]['saturation_signal_in_window'].values
            lightness_signal_in_window = landmark_df[bvp_mask]['lightness_signal_in_window'].values

            # 文字列をNumPy配列に変換
            r_signal_in_window = np.fromstring(r_signal_in_window[0].strip('[]'), sep=' ') if len(r_signal_in_window) > 0 else np.array([])
            g_signal_in_window = np.fromstring(g_signal_in_window[0].strip('[]'), sep=' ') if len(g_signal_in_window) > 0 else np.array([])
            b_signal_in_window = np.fromstring(b_signal_in_window[0].strip('[]'), sep=' ') if len(b_signal_in_window) > 0 else np.array([])
            saturation_signal_in_window = np.fromstring(saturation_signal_in_window[0].strip('[]'), sep=' ') if len(saturation_signal_in_window) > 0 else np.array([])
            lightness_signal_in_window = np.fromstring(lightness_signal_in_window[0].strip('[]'), sep=' ') if len(lightness_signal_in_window) > 0 else np.array([])

            print(f'    RGB信号の長さ: R={len(r_signal_in_window)}, G={len(g_signal_in_window)}, B={len(b_signal_in_window)}')
            
            # RGB信号が存在しない場合はスキップ
            if len(r_signal_in_window) == 0 or len(g_signal_in_window) == 0 or len(b_signal_in_window) == 0:
                print(f'    警告: RGB信号が存在しないため、landmark {landmark_id} をスキップします')
                continue
            
            # ============================================================================
            # Intensity計算(POS法ベース)
            # ============================================================================
            try:
                # RGB信号をCuPy配列に変換
                rgb_signal = np.array([[r_signal_in_window, g_signal_in_window, b_signal_in_window]], dtype=np.float32)
                rgb_cupy = cp.asarray(rgb_signal)
                
                # POS法のパラメータ
                eps = 10**-9
                X = rgb_cupy
                fps_cupy = cp.float32(fps)
                e, c, f = X.shape  # e = #estimators, c = 3 rgb ch., f = #frames
                w = int(1.6 * fps_cupy)  # window length

                # 固定の拍動ベクトル(論文の式30)
                u_pbv = cp.array([0.33, 0.77, 0.53], dtype=cp.float32)

                # 肌色調ベクトルを計算
                r_signal_square = r_signal_in_window ** 2
                g_signal_square = g_signal_in_window ** 2
                b_signal_square = b_signal_in_window ** 2
                norm = np.sqrt(r_signal_square + g_signal_square + b_signal_square)

                normalized_r = r_signal_in_window / norm
                normalized_g = g_signal_in_window / norm
                normalized_b = b_signal_in_window / norm

                skin_vector_array = np.array([normalized_r, normalized_g, normalized_b])
                skin_vector = cp.mean(cp.asarray(skin_vector_array), axis=1)

                # 標準化された肌色ベクトル
                u_skin = skin_vector / (cp.linalg.norm(skin_vector) + eps)
                
                # u_pbvとu_skinに直交するベクトルを求める
                v_n = cp.cross(u_skin, u_pbv)
                normalize_v_n = v_n / (cp.linalg.norm(v_n) + eps)

                # 投影行列P(1x3の行ベクトル)
                P = cp.reshape(normalize_v_n, (1, 3))

                # 初期化
                intensity_signal = cp.zeros((e, f))

                # スライディングウィンドウループ
                for n in cp.arange(w, f):
                    m = n - w + 1
                    
                    # 時間的正規化
                    Cn = X[:, :, m:(n + 1)]
                    M = 1.0 / (cp.mean(Cn, axis=2) + eps)
                    M_expanded = cp.expand_dims(M, axis=2)
                    Cn = cp.multiply(M_expanded, Cn)
                    
                    # 投影(uskinとupbvに直交する方向に投影してi(t)を抽出)
                    for estimator_idx in range(e):
                        projected = cp.dot(P, Cn[estimator_idx, :, :]).flatten()
                        # ゼロ平均化
                        projected = projected - cp.mean(projected)
                        # オーバーラップ加算
                        intensity_signal[estimator_idx, m:(n + 1)] = cp.add(
                            intensity_signal[estimator_idx, m:(n + 1)], 
                            projected
                        )
                
                # CuPy配列をNumPy配列に変換
                bvp_numpy = cp.asnumpy(intensity_signal)
                
                raw_bvp_signal = [bvp_numpy]
                bvp_signal = [bvp_numpy.copy()]

                # 後処理フィルタリング
                bvp_signal = vhr.BVP.apply_filter(
                    bvp_signal,
                    vhr.BVP.BPfilter,
                    params={'order': 6, 'minHz': 0.5, 'maxHz': 2.0, 'fps': fps}
                )
                bvp_signal = vhr.BVP.apply_filter(bvp_signal, vhr.BVP.zeromean)

                raw_bvp_signal_in_window = raw_bvp_signal[0] if len(raw_bvp_signal) > 0 else None
                filtered_bvp_signal_in_window = bvp_signal[0] if len(bvp_signal) > 0 else None

                # FFT解析
                raw_bvp_signal_in_window = raw_bvp_signal_in_window.flatten() if raw_bvp_signal_in_window is not None else None
                filtered_bvp_signal_in_window = filtered_bvp_signal_in_window.flatten()
                fft_result_dic = analyze_window_fft(filtered_bvp_signal_in_window, fps)

                # MAEの計算
                rppg_bpm = fft_result_dic['max_freq'] * 60
                bpm_MAE = np.abs(ecg_bpm_in_window_mean - rppg_bpm) if not np.isnan(ecg_bpm_in_window_mean) else np.nan

                print(f"    Intensity結果: ECG BPM={ecg_bpm_in_window_mean:.2f}, rPPG BPM={rppg_bpm:.2f}, MAE={bpm_MAE:.2f}")

                # 結果を保存
                window_info = {
                    'window_index': idx,
                    'landmark_id': landmark_id,
                    'window_size': window_size,
                    'stride': stride,
                    'frame_start': frame_start,
                    'frame_end': frame_end,
                    'window_start_time': window_start_time,
                    'window_end_time': window_end_time,
                    'r_signal_in_window': array_to_full_string(r_signal_in_window),
                    'g_signal_in_window': array_to_full_string(g_signal_in_window),
                    'b_signal_in_window': array_to_full_string(b_signal_in_window),
                    'raw_intensity_in_window': array_to_full_string(raw_bvp_signal_in_window) if raw_bvp_signal_in_window is not None else '',
                    'filtered_intensity_in_window': array_to_full_string(filtered_bvp_signal_in_window),
                    'ecg_bpm_in_window': array_to_full_string(ecg_bpm_in_window),
                    'ecg_bpm_mean': ecg_bpm_in_window_mean,
                    'rppg_bpm': rppg_bpm,
                    'bpm_MAE': bpm_MAE,
                    'max_freq': fft_result_dic['max_freq'],
                    'max_amplitude': fft_result_dic['max_amplitude'],
                    'spectral_centroid': fft_result_dic['spectral_centroid'],
                    'spectral_bandwidth': fft_result_dic['spectral_bandwidth'],
                    'total_power': fft_result_dic['total_power']
                }
                
                all_window_results.append(window_info)
                
            except Exception as e:
                print(f'    エラー: landmark {landmark_id}, window {idx} の処理中にエラーが発生しました: {e}')
                import traceback
                traceback.print_exc()
                continue
    
    # 全結果をDataFrameに変換して保存
    results_df = pd.DataFrame(all_window_results)
    results_save_dir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(results_save_dir, exist_ok=True)
    results_csv_path = os.path.join(results_save_dir, f'window_intensity_landmark_{dataName}.csv')
    results_df.to_csv(results_csv_path, index=False, encoding='utf-8-sig')
    print(f"\nIntensity結果をCSVに保存: {results_csv_path}")
    print(f"保存されたデータ件数: {len(results_df)}")

In [ ]:
holistic_rppg_results_dir = "rppgAccuracyEvalu"
SPLIT_TIME = 90

for i in range(len(movie_paths)):
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]
    print(f'Processing movie: {inputMoviePath}')
    
    # 動画のfpsを取得
    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    samplingRate = fps
    cap.release()
    
    # 肌検出RPPG結果の読み込み
    hol_POS_results_df = pd.read_csv(os.path.join(rootDir, holistic_rppg_results_dir, f'window_analysis_{dataName}.csv'))
    hol_POS_first_half_mask = hol_POS_results_df['window_end_time'] <= SPLIT_TIME
    hol_POS_second_half_mask = hol_POS_results_df['window_start_time'] >= SPLIT_TIME
    
    hol_POS_first_half_df = hol_POS_results_df[hol_POS_first_half_mask].copy()
    hol_POS_second_half_df = hol_POS_results_df[hol_POS_second_half_mask].copy()
    
    # landmarkごとの結果を読み込み
    landmark_window_analysis_path = os.path.join(rootDir, SAVE_DIR, f'{dataName}_window_analysis_{PATCH_SIZE}x{PATCH_SIZE}.csv')
    landmark_results_df = pd.read_csv(landmark_window_analysis_path)
    landmark_first_half_mask = landmark_results_df['window_end_time'] <= SPLIT_TIME
    landmark_second_half_mask = landmark_results_df['window_start_time'] >= SPLIT_TIME
    landmark_first_half_df = landmark_results_df[landmark_first_half_mask].copy()
    landmark_second_half_df = landmark_results_df[landmark_second_half_mask].copy()
    
    # 輝度信号ごとの結果を読み込み
    intensity_window_analysis_path = os.path.join(rootDir, SAVE_DIR, f'window_intensity_landmark_{dataName}.csv')
    intensity_results_df = pd.read_csv(intensity_window_analysis_path)
    intensity_first_half_mask = intensity_results_df['window_end_time'] <= SPLIT_TIME
    intensity_second_half_mask = intensity_results_df['window_start_time'] >= SPLIT_TIME
    intensity_first_half_df = intensity_results_df[intensity_first_half_mask].copy()
    intensity_second_half_df = intensity_results_df[intensity_second_half_mask].copy()
    
    # 図の保存先
    figure_dir = os.path.join(rootDir, SAVE_DIR, 'heatmaps')
    os.makedirs(figure_dir, exist_ok=True)
    
    # ============================================================================
    # 前半と後半それぞれでヒートマップを作成
    # ============================================================================
    for period, (landmark_df, intensity_df, hol_df) in [
        ('first_half', (landmark_first_half_df, intensity_first_half_df, hol_POS_first_half_df)),
        ('second_half', (landmark_second_half_df, intensity_second_half_df, hol_POS_second_half_df))
    ]:
        print(f'\n{period}のヒートマップを作成中...')
        
        # ============================================================================
        # 1. Landmarkベースの手法のヒートマップ
        # ============================================================================
        if len(landmark_df) > 0:
            # landmark_idを名前に変換
            landmark_df_copy = landmark_df.copy()
            landmark_df_copy['landmark_name'] = landmark_df_copy['landmark_id'].map(LANDMARK_ID_NAME_MAP)
            
            # window_sizeとlandmark_nameでグループ化してMAEの平均を計算
            pivot_data = landmark_df_copy.groupby(['window_size', 'landmark_name'])['bpm_MAE'].mean().reset_index()
            pivot_table = pivot_data.pivot(index='window_size', columns='landmark_name', values='bpm_MAE')
            
            # 列の順序を定義(視覚的に分かりやすい順序)
            desired_order = ["Forehead", "Upper Left Cheek", "Lower Left Cheek", 
                           "Upper Right Cheek", "Lower Right Cheek", "Nose", "Chin"]
            # 実際に存在する列のみを使用
            column_order = [col for col in desired_order if col in pivot_table.columns]
            pivot_table = pivot_table[column_order]
            
            # ヒートマップ作成
            fig, ax = plt.subplots(figsize=(14, 8))
            sns.heatmap(pivot_table, annot=True, fmt='.4f', cmap='YlOrRd', 
                       cbar_kws={'label': 'Mean MAE (BPM)'}, ax=ax)
            ax.set_xlabel('Landmark Region', fontsize=12)
            ax.set_ylabel('Window Size [s]', fontsize=12)
            ax.set_title(f'MAE vs Landmark Region and Window Size - {dataName} ({period})', fontsize=14)
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            
            output_path = os.path.join(figure_dir, f'heatmap_mae_landmark_{period}_{dataName}.png')
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            print(f'Landmarkヒートマップを保存: {output_path}')
            plt.close()
        
        # ============================================================================
        # 2. Intensityベースの手法のヒートマップ
        # ============================================================================
        if len(intensity_df) > 0:
            # landmark_idを名前に変換
            intensity_df_copy = intensity_df.copy()
            intensity_df_copy['landmark_name'] = intensity_df_copy['landmark_id'].map(LANDMARK_ID_NAME_MAP)
            
            # window_sizeとlandmark_nameでグループ化してMAEの平均を計算
            pivot_data_intensity = intensity_df_copy.groupby(['window_size', 'landmark_name'])['bpm_MAE'].mean().reset_index()
            pivot_table_intensity = pivot_data_intensity.pivot(index='window_size', columns='landmark_name', values='bpm_MAE')
            
            # 列の順序を定義
            column_order = [col for col in desired_order if col in pivot_table_intensity.columns]
            pivot_table_intensity = pivot_table_intensity[column_order]
            
            # ヒートマップ作成
            fig, ax = plt.subplots(figsize=(14, 8))
            sns.heatmap(pivot_table_intensity, annot=True, fmt='.4f', cmap='YlOrRd', 
                       cbar_kws={'label': 'Mean MAE (BPM)'}, ax=ax)
            ax.set_xlabel('Landmark Region', fontsize=12)
            ax.set_ylabel('Window Size [s]', fontsize=12)
            ax.set_title(f'MAE vs Landmark Region and Window Size (Intensity) - {dataName} ({period})', fontsize=14)
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            
            output_path = os.path.join(figure_dir, f'heatmap_mae_intensity_{period}_{dataName}.png')
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            print(f'Intensityヒートマップを保存: {output_path}')
            plt.close()
        
        # ============================================================================
        # 3. 肌検出ベースの手法のヒートマップ(BVP手法別)
        # ============================================================================
        if len(hol_df) > 0:
            # window_sizeとbvp_methodでグループ化してMAEの平均を計算
            pivot_data_hol = hol_df.groupby(['window_size', 'bvp_method'])['bpm_MAE'].mean().reset_index()
            pivot_table_hol = pivot_data_hol.pivot(index='window_size', columns='bvp_method', values='bpm_MAE')
            
            # ヒートマップ作成
            fig, ax = plt.subplots(figsize=(12, 8))
            sns.heatmap(pivot_table_hol, annot=True, fmt='.4f', cmap='YlOrRd', 
                       cbar_kws={'label': 'Mean MAE (BPM)'}, ax=ax)
            ax.set_xlabel('BVP Method', fontsize=12)
            ax.set_ylabel('Window Size [s]', fontsize=12)
            ax.set_title(f'MAE vs BVP Method and Window Size (Skin Detection) - {dataName} ({period})', fontsize=14)
            plt.tight_layout()
            
            output_path = os.path.join(figure_dir, f'heatmap_mae_skindetection_{period}_{dataName}.png')
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            print(f'Skin Detectionヒートマップを保存: {output_path}')
            plt.close()
        
        # ============================================================================
        # 4. 統合ヒートマップ(Landmark + Intensity + Skin Detection)
        # ============================================================================
        # 各手法の平均MAEを窓長ごとに計算
        combined_data = []
        
        # Landmarkベース(各landmarkの平均)
        if len(landmark_df) > 0:
            landmark_df_copy = landmark_df.copy()
            landmark_df_copy['landmark_name'] = landmark_df_copy['landmark_id'].map(LANDMARK_ID_NAME_MAP)
            
            for window_size in landmark_df_copy['window_size'].unique():
                window_data = landmark_df_copy[landmark_df_copy['window_size'] == window_size]
                for landmark_name in window_data['landmark_name'].unique():
                    landmark_mae = window_data[window_data['landmark_name'] == landmark_name]['bpm_MAE'].mean()
                    combined_data.append({
                        'window_size': window_size,
                        'method': f'LM_{landmark_name}',
                        'mae': landmark_mae
                    })
        
        # Intensityベース(各landmarkの平均)
        if len(intensity_df) > 0:
            intensity_df_copy = intensity_df.copy()
            intensity_df_copy['landmark_name'] = intensity_df_copy['landmark_id'].map(LANDMARK_ID_NAME_MAP)
            
            for window_size in intensity_df_copy['window_size'].unique():
                window_data = intensity_df_copy[intensity_df_copy['window_size'] == window_size]
                for landmark_name in window_data['landmark_name'].unique():
                    intensity_mae = window_data[window_data['landmark_name'] == landmark_name]['bpm_MAE'].mean()
                    combined_data.append({
                        'window_size': window_size,
                        'method': f'INT_{landmark_name}',
                        'mae': intensity_mae
                    })
        
        # Skin Detectionベース(各BVP手法)
        if len(hol_df) > 0:
            for window_size in hol_df['window_size'].unique():
                window_data = hol_df[hol_df['window_size'] == window_size]
                for bvp_method in window_data['bvp_method'].unique():
                    hol_mae = window_data[window_data['bvp_method'] == bvp_method]['bpm_MAE'].mean()
                    combined_data.append({
                        'window_size': window_size,
                        'method': f'SD_{bvp_method}',
                        'mae': hol_mae
                    })
        
        if len(combined_data) > 0:
            combined_df = pd.DataFrame(combined_data)
            pivot_combined = combined_df.pivot(index='window_size', columns='method', values='mae')
            
            # 列の順序を整理(Landmark -> Intensity -> Skin Detection)
            landmark_cols = [col for col in pivot_combined.columns if col.startswith('LM_')]
            intensity_cols = [col for col in pivot_combined.columns if col.startswith('INT_')]
            skin_cols = [col for col in pivot_combined.columns if col.startswith('SD_')]
            
            # 各グループ内でソート
            landmark_cols = sorted(landmark_cols, key=lambda x: desired_order.index(x.replace('LM_', '')) if x.replace('LM_', '') in desired_order else 999)
            intensity_cols = sorted(intensity_cols, key=lambda x: desired_order.index(x.replace('INT_', '')) if x.replace('INT_', '') in desired_order else 999)
            skin_cols = sorted(skin_cols)
            
            column_order = landmark_cols + intensity_cols + skin_cols
            pivot_combined = pivot_combined[column_order]
            
            # ヒートマップ作成
            fig, ax = plt.subplots(figsize=(24, 8))
            sns.heatmap(pivot_combined, annot=True, fmt='.4f', cmap='YlOrRd', 
                       cbar_kws={'label': 'Mean MAE (BPM)'}, ax=ax)
            ax.set_xlabel('Method (LM=Landmark, INT=Intensity, SD=Skin Detection)', fontsize=12)
            ax.set_ylabel('Window Size [s]', fontsize=12)
            ax.set_title(f'MAE vs Method and Window Size - All Methods Combined - {dataName} ({period})', fontsize=14)
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            
            output_path = os.path.join(figure_dir, f'heatmap_mae_combined_{period}_{dataName}.png')
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            print(f'統合ヒートマップを保存: {output_path}')
            plt.close()

print('\n全てのヒートマップ作成が完了しました。')

輝度信号を参考にした結果算出

In [ ]:
import cupy as cp
for i in range(len(movie_paths)):
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    print(f'Processing movie: {inputMoviePath}')

    # 動画のfpsを取得
    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    samplingRate = fps
    cap.release()

    save_dir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(save_dir, exist_ok=True)        

    # CSVファイルの読み込み
    ecg_csv_path = os.path.join(rootDir, dataName + '.csv')
    ecg_df = pd.read_csv(ecg_csv_path)

    ecg_RRI_csv_path = os.path.join(rootDir, f'RRI_Simple_{movie_names[i]}.csv')
    ecg_RRI_df = pd.read_csv(ecg_RRI_csv_path)

    # landmarkで共通のecg_bpm_in_window_meanを取得
    ecg_bpm_in_window = ecg_RRI_df['BPM']
    ecg_bpm_in_window_mean = ecg_RRI_df['BPM'].values.mean()

    # ROIデータの読み込み
    os.makedirs(save_dir, exist_ok=True)
    window_signals_data_path = os.path.join(rootDir, SAVE_DIR, f'{dataName}_{PATCH_SIZE}x{PATCH_SIZE}_ROI_window_signals.csv')
    window_signals_data_df = pd.read_csv(window_signals_data_path)

    stride_segment_calculator = StrideSegmentCalculator(window_sizes=WINDOW_SIZES, strides=STRIDES)
    analysis_df = stride_segment_calculator.create_analysis_dataframe(total_frames, fps)

    # ランドマークごとに実行
    # 結果を格納するリスト
    all_window_results = []
    
    for idx, row in analysis_df.iterrows():
        window_size = row['window_size']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        stride = row['stride']

        # 窓の時間範囲を計算
        window_start_time = frame_start / fps
        window_end_time = frame_end / fps

        if(window_end_time > total_frames / fps):
            continue

        print(f"\nウィンドウ {idx}: 窓サイズ {window_size}s, ストライド {stride}s, フレーム {frame_start}-{frame_end}, 時間 {window_start_time:.2f}-{window_end_time:.2f}s")

        # 該当する窓の時間範囲内の真値RRIデータを抽出
        ecg_RRI_mask = (ecg_RRI_df['time'] >= window_start_time) & \
                (ecg_RRI_df['time'] < window_end_time)
        ecg_bpm_in_window = ecg_RRI_df[ecg_RRI_mask]['BPM'].values
        ecg_bpm_in_window_mean = np.mean(ecg_bpm_in_window) if len(ecg_bpm_in_window) > 0 else np.nan
        print(f'    True Value RRI count in window: {len(ecg_bpm_in_window)}, Mean BPM: {ecg_bpm_in_window_mean:.2f}')
        
        for landmark_id in window_signals_data_df['landmark_id'].unique():
            print(f'  Analyzing landmark_id: {landmark_id}')
            landmark_df = window_signals_data_df[window_signals_data_df['landmark_id'] == landmark_id]

            # 該当する窓の時間範囲内のRGB信号を抽出し、窓内でBVP算出
            bvp_mask = landmark_df['window_index'] == idx

            r_signal_in_window = landmark_df[bvp_mask]['r_signal_in_window'].values
            g_signal_in_window = landmark_df[bvp_mask]['g_signal_in_window'].values
            b_signal_in_window = landmark_df[bvp_mask]['b_signal_in_window'].values
            saturation_signal_in_window = landmark_df[bvp_mask]['saturation_signal_in_window'].values
            lightness_signal_in_window = landmark_df[bvp_mask]['lightness_signal_in_window'].values

            # 文字列をNumPy配列に変換
            r_signal_in_window = np.fromstring(r_signal_in_window[0].strip('[]'), sep=' ') if len(r_signal_in_window) > 0 else np.array([])
            g_signal_in_window = np.fromstring(g_signal_in_window[0].strip('[]'), sep=' ') if len(g_signal_in_window) > 0 else np.array([])
            b_signal_in_window = np.fromstring(b_signal_in_window[0].strip('[]'), sep=' ') if len(b_signal_in_window) > 0 else np.array([])
            saturation_signal_in_window = np.fromstring(saturation_signal_in_window[0].strip('[]'), sep=' ') if len(saturation_signal_in_window) > 0 else np.array([])
            lightness_signal_in_window = np.fromstring(lightness_signal_in_window[0].strip('[]'), sep=' ') if len(lightness_signal_in_window) > 0 else np.array([])

            print(f'    RGB信号の長さ: R={len(r_signal_in_window)}, G={len(g_signal_in_window)}, B={len(b_signal_in_window)}')
            
            # RGB信号が存在しない場合はスキップ
            if len(r_signal_in_window) == 0 or len(g_signal_in_window) == 0 or len(b_signal_in_window) == 0:
                print(f'    警告: RGB信号が存在しないため、landmark {landmark_id} をスキップします')
                continue
            
            # RGB信号をCuPy配列に変換
            rgb_signal = np.array([[r_signal_in_window, g_signal_in_window, b_signal_in_window]], dtype=np.float32)

            # CuPy配列に変換
            rgb_cupy = cp.asarray(rgb_signal)

            # POS法のパラメータ
            eps = 10**-9
            X = rgb_cupy
            fps_cupy = cp.float32(fps)
            e, c, f = X.shape  # e = #estimators, c = 3 rgb ch., f = #frames
            w = int(1.6 * fps_cupy)  # window length

            # 投影行列P(現在のパターンを使用)
            P_cupy = cp.asarray(np.array([[0.0, 1.0, -1.0],
                                         [-1.0, 1.0, 1.0]], dtype=np.float32))
            Q = cp.stack([P_cupy for _ in range(e)], axis=0)

            # 初期化
            H = cp.zeros((e, f))

            # 診断情報を保存するリスト
            alpha_list = []
            M_list = []
            S1_list = []
            S2_list = []
            alpha_S2_list = []
            window_indices = []

            # スライディングウィンドウループ
            for n in cp.arange(w, f):
                m = n - w + 1
                
                # 時間的正規化
                Cn = X[:, :, m:(n + 1)]
                M = 1.0 / (cp.mean(Cn, axis=2) + eps)
                M_expanded = cp.expand_dims(M, axis=2)
                Cn = cp.multiply(M_expanded, Cn)
                
                # Mの値を保存
                M_list.append(cp.asnumpy(M))
                
                # 投影
                S = cp.dot(Q, Cn)
                S = S[0, :, :, :]
                S = cp.swapaxes(S, 0, 1)
                
                # チューニング
                S1 = S[:, 0, :]
                S2 = S[:, 1, :]
                alpha = cp.std(S1, axis=1) / (eps + cp.std(S2, axis=1))
                
                # S1とS2を保存
                S1_list.append(cp.asnumpy(S1))
                S2_list.append(cp.asnumpy(S2))
                
                # alphaの値を保存
                alpha_list.append(cp.asnumpy(alpha))
                
                alpha_expanded = cp.expand_dims(alpha, axis=1)
                alpha_S2 = alpha_expanded * S2
                
                # alpha*S2を保存
                alpha_S2_list.append(cp.asnumpy(alpha_S2))
                
                window_indices.append(int(n))
                
                Hn = cp.add(S1, alpha_S2)
                Hnm = Hn - cp.expand_dims(cp.mean(Hn, axis=1), axis=1)
                
                # オーバーラップ加算
                H[:, m:(n + 1)] = cp.add(H[:, m:(n + 1)], Hnm)
            
                
            # CuPy配列をNumPy配列に変換
            bvp_cupy = H
            bvp_numpy = cp.asnumpy(bvp_cupy)

            raw_bvp_signal = [bvp_numpy]
            bvp_signal = [bvp_numpy.copy()]

            # 後処理フィルタリング
            bvp_signal = vhr.BVP.apply_filter(
                bvp_signal,
                vhr.BVP.BPfilter,
                params={'order': 6, 'minHz': 0.5, 'maxHz': 2.0, 'fps': fps}
            )
            bvp_signal = vhr.BVP.apply_filter(bvp_signal, vhr.BVP.zeromean)

            raw_bvp_signal_in_window = raw_bvp_signal[0] if len(raw_bvp_signal) > 0 else None
            filtered_bvp_signal_in_window = bvp_signal[0] if len(bvp_signal) > 0 else None

            # FFT解析
            raw_bvp_signal_in_window = raw_bvp_signal_in_window.flatten() if raw_bvp_signal_in_window is not None else None
            filtered_bvp_signal_in_window = filtered_bvp_signal_in_window.flatten()
            fft_result_dic = analyze_window_fft(filtered_bvp_signal_in_window, fps)

            # MAEの計算
            rppg_bpm = fft_result_dic['max_freq'] * 60
            bpm_MAE = np.abs(ecg_bpm_in_window_mean - rppg_bpm) if not np.isnan(ecg_bpm_in_window_mean) else np.nan

            # 結果を保存
            window_info = {
                'window_index': idx,
                'landmark_id': landmark_id,
                'window_size': window_size,
                'stride': stride,
                'frame_start': frame_start,
                'frame_end': frame_end,
                'window_start_time': window_start_time,
                'window_end_time': window_end_time,
                'r_signal_in_window': array_to_full_string(r_signal_in_window),
                'g_signal_in_window': array_to_full_string(g_signal_in_window),
                'b_signal_in_window': array_to_full_string(b_signal_in_window),
                'raw_bvp_in_window': array_to_full_string(raw_bvp_signal_in_window) if raw_bvp_signal_in_window is not None else '',
                'filtered_bvp_in_window': array_to_full_string(filtered_bvp_signal_in_window),
                'ecg_bpm_in_window': array_to_full_string(ecg_bpm_in_window),
                'ecg_bpm_mean': ecg_bpm_in_window_mean,
                'rppg_bpm': rppg_bpm,
                'bpm_MAE': bpm_MAE,
                'max_freq': fft_result_dic['max_freq'],
                'max_amplitude': fft_result_dic['max_amplitude'],
                'spectral_centroid': fft_result_dic['spectral_centroid'],
                'spectral_bandwidth': fft_result_dic['spectral_bandwidth'],
                'total_power': fft_result_dic['total_power']
            }
            
            all_window_results.append(window_info)

    # 全結果をDataFrameに変換して保存
    results_df = pd.DataFrame(all_window_results)
    results_save_dir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(results_save_dir, exist_ok=True)
    results_csv_path = os.path.join(results_save_dir, f'{dataName}_window_analysis_advanced_POS_{PATCH_SIZE}x{PATCH_SIZE}.csv')
    results_df.to_csv(results_csv_path, index=False, encoding='utf-8-sig')
    print(f"\nBVP結果をCSVに保存: {results_csv_path}")
    print(f"保存されたデータ件数: {len(results_df)}")

In [ ]:
holistic_rppg_results_dir = "rppgAccuracyEvalu"
SPLIT_TIME = 90

for i in range(len(movie_paths)):
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]
    print(f'Processing movie: {inputMoviePath}')
    
    # 動画のfpsを取得
    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    samplingRate = fps
    cap.release()
    
    # 肌検出RPPG結果の読み込み
    hol_POS_results_df = pd.read_csv(os.path.join(rootDir, holistic_rppg_results_dir, f'window_analysis_{dataName}.csv'))
    hol_POS_first_half_mask = hol_POS_results_df['window_end_time'] <= SPLIT_TIME
    hol_POS_second_half_mask = hol_POS_results_df['window_start_time'] >= SPLIT_TIME
    
    hol_POS_first_half_df = hol_POS_results_df[hol_POS_first_half_mask].copy()
    hol_POS_second_half_df = hol_POS_results_df[hol_POS_second_half_mask].copy()
    
    # landmarkごとの結果を読み込み
    landmark_window_analysis_path = os.path.join(rootDir, SAVE_DIR, f'{dataName}_window_analysis_advanced_POS_{PATCH_SIZE}x{PATCH_SIZE}.csv')
    landmark_results_df = pd.read_csv(landmark_window_analysis_path)
    landmark_first_half_mask = landmark_results_df['window_end_time'] <= SPLIT_TIME
    landmark_second_half_mask = landmark_results_df['window_start_time'] >= SPLIT_TIME
    landmark_first_half_df = landmark_results_df[landmark_first_half_mask].copy()
    landmark_second_half_df = landmark_results_df[landmark_second_half_mask].copy()
    
    # 輝度信号ごとの結果を読み込み
    intensity_window_analysis_path = os.path.join(rootDir, SAVE_DIR, f'window_intensity_landmark_{dataName}.csv')
    intensity_results_df = pd.read_csv(intensity_window_analysis_path)
    intensity_first_half_mask = intensity_results_df['window_end_time'] <= SPLIT_TIME
    intensity_second_half_mask = intensity_results_df['window_start_time'] >= SPLIT_TIME
    intensity_first_half_df = intensity_results_df[intensity_first_half_mask].copy()
    intensity_second_half_df = intensity_results_df[intensity_second_half_mask].copy()
    
    # 図の保存先
    figure_dir = os.path.join(rootDir, SAVE_DIR, 'heatmaps')
    os.makedirs(figure_dir, exist_ok=True)
    
    # ============================================================================
    # 前半と後半それぞれでヒートマップを作成
    # ============================================================================
    for period, (landmark_df, intensity_df, hol_df) in [
        ('first_half', (landmark_first_half_df, intensity_first_half_df, hol_POS_first_half_df)),
        ('second_half', (landmark_second_half_df, intensity_second_half_df, hol_POS_second_half_df))
    ]:
        print(f'\n{period}のヒートマップを作成中...')
        
        # ============================================================================
        # 1. Landmarkベースの手法のヒートマップ
        # ============================================================================
        if len(landmark_df) > 0:
            # landmark_idを名前に変換
            landmark_df_copy = landmark_df.copy()
            landmark_df_copy['landmark_name'] = landmark_df_copy['landmark_id'].map(LANDMARK_ID_NAME_MAP)
            
            # window_sizeとlandmark_nameでグループ化してMAEの平均を計算
            pivot_data = landmark_df_copy.groupby(['window_size', 'landmark_name'])['bpm_MAE'].mean().reset_index()
            pivot_table = pivot_data.pivot(index='window_size', columns='landmark_name', values='bpm_MAE')
            
            # 列の順序を定義(視覚的に分かりやすい順序)
            desired_order = ["Forehead", "Upper Left Cheek", "Lower Left Cheek", 
                           "Upper Right Cheek", "Lower Right Cheek", "Nose", "Chin"]
            # 実際に存在する列のみを使用
            column_order = [col for col in desired_order if col in pivot_table.columns]
            pivot_table = pivot_table[column_order]
            
            # ヒートマップ作成
            fig, ax = plt.subplots(figsize=(14, 8))
            sns.heatmap(pivot_table, annot=True, fmt='.4f', cmap='YlOrRd', 
                       cbar_kws={'label': 'Mean MAE (BPM)'}, ax=ax)
            ax.set_xlabel('Landmark Region', fontsize=12)
            ax.set_ylabel('Window Size [s]', fontsize=12)
            ax.set_title(f'MAE vs Landmark Region and Window Size - {dataName} ({period})', fontsize=14)
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            
            output_path = os.path.join(figure_dir, f'heatmap_mae_landmark_{period}_{dataName}_adaptive.png')
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            print(f'Landmarkヒートマップを保存: {output_path}')
            plt.close()
        
        # ============================================================================
        # 2. Intensityベースの手法のヒートマップ
        # ============================================================================
        if len(intensity_df) > 0:
            # landmark_idを名前に変換
            intensity_df_copy = intensity_df.copy()
            intensity_df_copy['landmark_name'] = intensity_df_copy['landmark_id'].map(LANDMARK_ID_NAME_MAP)
            
            # window_sizeとlandmark_nameでグループ化してMAEの平均を計算
            pivot_data_intensity = intensity_df_copy.groupby(['window_size', 'landmark_name'])['bpm_MAE'].mean().reset_index()
            pivot_table_intensity = pivot_data_intensity.pivot(index='window_size', columns='landmark_name', values='bpm_MAE')
            
            # 列の順序を定義
            column_order = [col for col in desired_order if col in pivot_table_intensity.columns]
            pivot_table_intensity = pivot_table_intensity[column_order]
            
            # ヒートマップ作成
            fig, ax = plt.subplots(figsize=(14, 8))
            sns.heatmap(pivot_table_intensity, annot=True, fmt='.4f', cmap='YlOrRd', 
                       cbar_kws={'label': 'Mean MAE (BPM)'}, ax=ax)
            ax.set_xlabel('Landmark Region', fontsize=12)
            ax.set_ylabel('Window Size [s]', fontsize=12)
            ax.set_title(f'MAE vs Landmark Region and Window Size (Intensity) - {dataName} ({period})', fontsize=14)
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            
            output_path = os.path.join(figure_dir, f'heatmap_mae_intensity_{period}_{dataName}_adaptive.png')
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            print(f'Intensityヒートマップを保存: {output_path}')
            plt.close()
        
        # ============================================================================
        # 3. 肌検出ベースの手法のヒートマップ(BVP手法別)
        # ============================================================================
        if len(hol_df) > 0:
            # window_sizeとbvp_methodでグループ化してMAEの平均を計算
            pivot_data_hol = hol_df.groupby(['window_size', 'bvp_method'])['bpm_MAE'].mean().reset_index()
            pivot_table_hol = pivot_data_hol.pivot(index='window_size', columns='bvp_method', values='bpm_MAE')
            
            # ヒートマップ作成
            fig, ax = plt.subplots(figsize=(12, 8))
            sns.heatmap(pivot_table_hol, annot=True, fmt='.4f', cmap='YlOrRd', 
                       cbar_kws={'label': 'Mean MAE (BPM)'}, ax=ax)
            ax.set_xlabel('BVP Method', fontsize=12)
            ax.set_ylabel('Window Size [s]', fontsize=12)
            ax.set_title(f'MAE vs BVP Method and Window Size (Skin Detection) - {dataName} ({period})', fontsize=14)
            plt.tight_layout()
            
            output_path = os.path.join(figure_dir, f'heatmap_mae_skindetection_{period}_{dataName}_adaptive.png')
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            print(f'Skin Detectionヒートマップを保存: {output_path}')
            plt.close()
        
        # ============================================================================
        # 4. 統合ヒートマップ(Landmark + Intensity + Skin Detection)
        # ============================================================================
        # 各手法の平均MAEを窓長ごとに計算
        combined_data = []
        
        # Landmarkベース(各landmarkの平均)
        if len(landmark_df) > 0:
            landmark_df_copy = landmark_df.copy()
            landmark_df_copy['landmark_name'] = landmark_df_copy['landmark_id'].map(LANDMARK_ID_NAME_MAP)
            
            for window_size in landmark_df_copy['window_size'].unique():
                window_data = landmark_df_copy[landmark_df_copy['window_size'] == window_size]
                for landmark_name in window_data['landmark_name'].unique():
                    landmark_mae = window_data[window_data['landmark_name'] == landmark_name]['bpm_MAE'].mean()
                    combined_data.append({
                        'window_size': window_size,
                        'method': f'LM_{landmark_name}',
                        'mae': landmark_mae
                    })
        
        # Intensityベース(各landmarkの平均)
        if len(intensity_df) > 0:
            intensity_df_copy = intensity_df.copy()
            intensity_df_copy['landmark_name'] = intensity_df_copy['landmark_id'].map(LANDMARK_ID_NAME_MAP)
            
            for window_size in intensity_df_copy['window_size'].unique():
                window_data = intensity_df_copy[intensity_df_copy['window_size'] == window_size]
                for landmark_name in window_data['landmark_name'].unique():
                    intensity_mae = window_data[window_data['landmark_name'] == landmark_name]['bpm_MAE'].mean()
                    combined_data.append({
                        'window_size': window_size,
                        'method': f'INT_{landmark_name}',
                        'mae': intensity_mae
                    })
        
        # Skin Detectionベース(各BVP手法)
        if len(hol_df) > 0:
            for window_size in hol_df['window_size'].unique():
                window_data = hol_df[hol_df['window_size'] == window_size]
                for bvp_method in window_data['bvp_method'].unique():
                    hol_mae = window_data[window_data['bvp_method'] == bvp_method]['bpm_MAE'].mean()
                    combined_data.append({
                        'window_size': window_size,
                        'method': f'SD_{bvp_method}',
                        'mae': hol_mae
                    })
        
        if len(combined_data) > 0:
            combined_df = pd.DataFrame(combined_data)
            pivot_combined = combined_df.pivot(index='window_size', columns='method', values='mae')
            
            # 列の順序を整理(Landmark -> Intensity -> Skin Detection)
            landmark_cols = [col for col in pivot_combined.columns if col.startswith('LM_')]
            intensity_cols = [col for col in pivot_combined.columns if col.startswith('INT_')]
            skin_cols = [col for col in pivot_combined.columns if col.startswith('SD_')]
            
            # 各グループ内でソート
            landmark_cols = sorted(landmark_cols, key=lambda x: desired_order.index(x.replace('LM_', '')) if x.replace('LM_', '') in desired_order else 999)
            intensity_cols = sorted(intensity_cols, key=lambda x: desired_order.index(x.replace('INT_', '')) if x.replace('INT_', '') in desired_order else 999)
            skin_cols = sorted(skin_cols)
            
            column_order = landmark_cols + intensity_cols + skin_cols
            pivot_combined = pivot_combined[column_order]
            
            # ヒートマップ作成
            fig, ax = plt.subplots(figsize=(24, 8))
            sns.heatmap(pivot_combined, annot=True, fmt='.4f', cmap='YlOrRd', 
                       cbar_kws={'label': 'Mean MAE (BPM)'}, ax=ax)
            ax.set_xlabel('Method (LM=Landmark, INT=Intensity, SD=Skin Detection)', fontsize=12)
            ax.set_ylabel('Window Size [s]', fontsize=12)
            ax.set_title(f'MAE vs Method and Window Size - All Methods Combined - {dataName} ({period})', fontsize=14)
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            
            output_path = os.path.join(figure_dir, f'heatmap_mae_combined_{period}_{dataName}_adaptive.png')
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            print(f'統合ヒートマップを保存: {output_path}')
            plt.close()

print('\n全てのヒートマップ作成が完了しました。')